# Experiment 05-01

In [1]:
import numpy as np
import hashlib
from binascii import hexlify, unhexlify
import pyopencl as cl
from Library.opencl_information import opencl_information

## Show the available Platforms

In [2]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:25)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
 Device - Type: ALL | CPU
 Device - Max Clock Speed: 2300 Mhz
 Device - Compute Units: 16
 Device - Local Memory: 32 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 16 GB
 Device - Max Buffer/Image Size: 4096 MB
 Device - Max Work Group Size: 1024


 --------------------------------------------------------
 Device - Name: Intel(R) UHD Graphics 630
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1200 Mhz
 Device - Compute Units: 24
 Device - Local Memory: 64 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 2 GB
 Device - Max Buffer/Image Size: 384 MB
 Device - Max Work Group Size: 256


 --------------------------------------------------------
 Device - Name: AMD Radeon Pro 

## Configure the OpenCL Context

In [3]:
platform_number = 0
device_number = 2

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [4]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [5]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: get_single_hash;get_random_string;get_single_hash_nonce;mine_eight;mine_nonce;hash_main;get_random_numbers


/Users/foulds/env/dsm070/lib/python3.8/site-packages/pyopencl/__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


## Initial Tests

In [6]:
from hashlib import sha256
from binascii import hexlify
inputs = [
b"FOO-0x000000003B1BD2039" + b" " * 53 + b"\xdd\x3f\xf4\x6f",
b"BAR-0x00000000307238E22" + b" " * 53 + b"\xa8\x0d\xa3\x23",
b"FOOBAR-0x000000001BB6C4C9F" + b" " * 50 + b"\xb0\x1d\x7c\x21"
]
h = [sha256(sha256(x).digest()).digest() for x in inputs]
xor = bytes([h[0][i] ^ h[1][i] ^ h[2][i] for i in range(32)])
print(' sha256({})'.format(sha256(inputs[0]).hexdigest()))
print('^ sha256({})'.format(sha256(inputs[1]).hexdigest()))
print('^ sha256({})'.format(sha256(inputs[2]).hexdigest()))
print(' =====================================================')
print(' {}'.format(hexlify(xor).decode()))

 sha256(2143d3c4d879d7497c8ca4690543ac99ec9fee4fd0aa3f1f7561f88cf0b0f92c)
^ sha256(c0bb10795c2df1c79c4f952db2fee96454f49f559b3dd5fc73ff95420a24c9b9)
^ sha256(cadc1b67d589015b79cecc811b22e2f0b52334600832580165b4cee63e15f7d0)
 ccb9cef8d7d1c0c67d81536695eeff7e00000000000000000000000000000000


### Single Hash

In [8]:
# the plaintext to hash
#plaintext = 'bells hel the quick brown fox jumps over the lazy dog 12356'
#plaintext = '665782'
#plaintext = 'this is a description of the latest block' + '395707976'
#plaintext = '120Make America Great Again'
plaintext = 'this is a description of the latest block*jTi0@!N{oOaQf'
#plaintext = '69817Make America Great Again'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

# the hash output
hash_output = np.zeros(8, dtype=np.uint32)
leading_zeros = np.zeros(1, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

cl_leading_zeros = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    leading_zeros.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output,
    cl_leading_zeros)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)
cl.enqueue_copy(cl_queue, leading_zeros, cl_leading_zeros)

# print the results
print("Plaintext: %s" % plaintext)
print("Leading Zeros: %d" % leading_zeros[0])
print("CL result      : %s" % hexlify(hash_output))
print("Correct result : %s" % hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest()))
print("CL output      : %s" % hash_output)
print("CL output      : %s" % np.frombuffer(hash_output, dtype=np.uint8))
print("CL output      : %s" % hash_output.tobytes())

assert hexlify(hash_output) == hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest())


Plaintext: this is a description of the latest block*jTi0@!N{oOaQf
Leading Zeros: 8
CL result      : b'000000000a71c29b450284efe6325204d4f3b8867a25db00373fe5b90b0e8f57'
Correct result : b'000000000a71c29b450284efe6325204d4f3b8867a25db00373fe5b90b0e8f57'
CL output      : [         0 2613211402 4018405957   72495846 2260267988   14361978
 3118808887 1468993035]
CL output      : [  0   0   0   0  10 113 194 155  69   2 132 239 230  50  82   4 212 243
 184 134 122  37 219   0  55  63 229 185  11  14 143  87]
CL output      : b'\x00\x00\x00\x00\nq\xc2\x9bE\x02\x84\xef\xe62R\x04\xd4\xf3\xb8\x86z%\xdb\x007?\xe5\xb9\x0b\x0e\x8fW'


### Random String Generation

In [23]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
start = np.uint8(32)
end = np.uint8(126)
length = np.uint32(4)
result = np.zeros(length, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_start = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=start)
cl_end = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=end)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=length)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)

# execute the program
cl_program.get_random_numbers(
    cl_queue, (1,), None,
    cl_seed,
    cl_start,
    cl_end,
    cl_len,
    cl_result)

# get the results
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Seed: {seed}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Seed: 2101148060
Result: [ 74 108  47 113]
Result String: b'Jl/q'


In [24]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
result = np.zeros(shape=16, dtype=np.uint8)
length = np.zeros(shape=(1,), dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, length.nbytes)

# execute the program
cl_program.get_random_string(
    cl_queue, (1,), None,
    cl_seed,
    cl_result,
    cl_len)

# get the results
cl.enqueue_copy(cl_queue, length, cl_len)
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Length: {length}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Length: [13]
Result: [107  62  58 108  53 103  73 123  68 118 114  69 109   0 255  59]
Result String: b'k>:l5gI{DvrEm\x00\xff;'


## Test get_single_hash_nonce

In [25]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
nonce = np.zeros(shape=16, dtype=np.uint8)
nonce_len = np.zeros(shape=1, dtype=np.uint8)
hash = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
cl_hash_output = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, hash_output.nbytes)

# execute the program
cl_program.get_single_hash_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

nonce_str = nonce[:nonce_len[0]].tobytes().decode('UTF-8')

# print the results
print("Seed: %s" % seed)
print("Plaintext: %s" % plaintext)
print("Nonce: %s" % nonce)
print("Nonce Length: %s" % nonce_len[0])
print("Nonce String: %s" % nonce_str)
print("Hash Output: %s" % hash_output)
print("Hash Output string: %s" % hexlify(hash_output))
#print("No Nonce          : %s" % hexlify(hashlib.sha256((plaintext).encode('utf-8')).digest()))
print("Correct result    : %s" % hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest()))

assert hexlify(hash_output) == hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest())


Seed: 1015571135
Plaintext: this is a description of the latest block
Nonce: [ 55  73 123  90   0   0   0   0   0   0   0   0   0   0   0   0]
Nonce Length: 4
Nonce String: 7I{Z
Hash Output: [1476689860 2603870842 1383715013 2497883987 4276640420  264520075
 3754695946 2144851127]
Hash Output string: b'c47f04587aea339bc5d0795253afe294a45ae8fe8b41c40f0a1dccdfb7d4d77f'
Correct result    : b'c47f04587aea339bc5d0795253afe294a45ae8fe8b41c40f0a1dccdfb7d4d77f'


## Mine Nonce

In [26]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(500000)
nonce = np.zeros(shape=16 * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_nonce(
        cl_queue, (1,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len)

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

    # interpret the results
    for i in range(0, 64):
        if nonce_len[i] > 0:
            if (i not in zeros_found):
                nonce_str = nonce[i * 16:i * 16 + nonce_len[i]].tobytes().decode('UTF-8')
                zeros_found[i] = nonce_str
                hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

                print("%4d: [%2d] %16s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

    keep_running = False

   1: [11]      +Moe(Jl[}@b 031ea04cb7b51d8ad30adf257c5e2a71f647db0bbc3dc0d65e0c131cd5192fcd
   2: [10]       "o=_-zHU#p 00ae9ddbab577dfaf3973a9afd724c98ac666b5546954125a69961cac967558b
   3: [ 5]            7b5]0 000441de0caf81c9912a40a9cbee83a9d76ff41cba6d311c0cf7f4ac34909596
   4: [10]       oKxFs"O|J_ 000066085472838f2f3c4992e930023f325811226055a35154eea6cd84bc6632
